In [ ]:
# -*- coding: UTF-8 -*-

In [10]:
import numpy as np
import math
import re
import pandas as pd
from bs4 import BeautifulSoup
#from google.colab import drive
#import zipfile
#import seaborn as sns
#import spacy as sp
import string
import random
import matplotlib.pyplot as plt
import nltk
nltk.download('punkt')
from textblob import TextBlob

[nltk_data] Downloading package punkt to /home/pcorrea/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
#.mount("/content/drive")

In [19]:
cols = ['title','reviewId', 'restaurantId','author','reviewBody','language','mobileDevice','userLocation','publicationDate','visitDate','ratingValue','itemReviewed','address','addressLocality','addressRegion','postalCode','country','website','email','phoneNumber','averageRating']
train_data = pd.read_csv('pc-dataset-v2.dat', names= cols, engine = 'python', delimiter=',', skiprows=0)
train_data.describe()

,ratingValue,averageRating
count,100000.00000,100000.000000
mean,4.25719,4.254855
std,0.96880,0.400535
min,1.00000,1.000000
25%,4.00000,4.000000
50%,5.00000,4.500000
75%,5.00000,4.500000
max,5.00000,5.000000


In [20]:
train_data.loc[pd.isnull(train_data['reviewBody'])]

,title,reviewId,restaurantId,author,reviewBody,language,mobileDevice,userLocation,publicationDate,visitDate,...,itemReviewed,address,addressLocality,addressRegion,postalCode,country,website,email,phoneNumber,averageRating


In [21]:
train_data.head()

,title,reviewId,restaurantId,author,reviewBody,language,mobileDevice,userLocation,publicationDate,visitDate,...,itemReviewed,address,addressLocality,addressRegion,postalCode,country,website,email,phoneNumber,averageRating
0,Jantar,692856288,2344932,larinhavercosa222,Comida maravilhosa. lugar do verdadeiro chicle...,pt,True,São Carlos. SP,26/07/2019,07/2019,...,Imperador dos Camar\u00F5es,Avenida Doutor Ant\u00F4nio Gouveia,NaN,Alagoas,57030-170,Brasil,http://www.imperadordoscamaroes.com.br,contato@imperadordoscamaroes.com.br,+55 82 3231-4134,4.5
1,Uma boa experi\u00EAncia.,229208487,3720906,Pano P,Comida boa. drinks legais e um ambiente bem ac...,pt,True,Manaus. AM,16/09/2014,None,...,Guacamole Natal,Avenida Praia de Ponta Negra 9068 Ponta Negra,NaN,Rio Grande do Norte,59092-100,Brasil,http://www.facebook.com/GuacamoleNatal,guacamolenatal@gmail.com,+55 84 3219-3646,4.5
2,Otimo self service,679002332,2248105,Magma100,Excelente pra ir com a fam\u00EDlia! \u00D3tim...,pt,True,Goiânia. GO,04/06/2019,07/2018,...,Chica Pitanga,Rua Petrolina 19,Recife,Pernambuco,51021-250,Brasil,http://www.chicapitanga.com.br,chicapitanga@chicapitanga.com.br,+55 81 3334-8260,4.5
3,O sorvete mais tradicional do estado,355671519,8168302,Yurie M,Tomo sorvete na Visabor faz mais de 30 anos. S...,pt,False,Aracaju. SE,15/03/2016,01/2016,...,Sorvetes Vi Sabor,Avenida Francisco Porto 59 Grageru,Aracaju,Sergipe,NaN,Brasil,None,None,None,4.0
4,Melhor hamburger!,268741921,5395141,Debora_C85,O melhor hamburger que j\u00E1 comi! Excelente...,pt,False,Rio de Janeiro. RJ,28/04/2015,04/2015,...,Z Deli Sandwich Shop,Rua Haddock Lobo 1386 Jardins,S\u00E3o Paulo,Estado de S\u00E3o Paulo,01414-002,Brasil,http://www.facebook.com/zdelisandwichshop/,None,551130830021,4.5


In [22]:
train_data.tail()

,title,reviewId,restaurantId,author,reviewBody,language,mobileDevice,userLocation,publicationDate,visitDate,...,itemReviewed,address,addressLocality,addressRegion,postalCode,country,website,email,phoneNumber,averageRating
99995,Nova op\u00E7\u00E3o no Paseo!!!,672210394,17424548,Fabiano B,Ainda n\u00E3o inaugurado oficialmente. est\u0...,pt,True,Porto Alegre. RS,07/05/2019,05/2019,...,Boteco Maestro,Avenida Wenceslau Escobar 1823,Porto Alegre,Rio Grande do Sul,91900-000,Brasil,http://www.facebook.com/botecomaestro,None,+55 51 99353-6602,4.0
99996,Bom,380904400,2440184,Chris L,Bom atendimento. lugar bonito. comida muito go...,pt,True,Cork. Irlanda,08/06/2016,01/2016,...,Casa do Marqu\u00EAs,Rua Marques do Pombal. 1814 Higien\u00F3polis,Porto Alegre,Rio Grande do Sul,90540-000,Brasil,http://galeteriacasadomarques.com.br/,galeteriacasadomarques@galeteriacasadomarques....,+55 51 3343-4303,4.5
99997,Bom bar,186111858,2392996,Guilherme P,O Rio Scenarium \u00E9 uma das baladas mais fa...,pt,False,São Paulo. SP,27/11/2013,11/2013,...,Rio Scenarium,Rua do Lavradio 20 Centro,Rio de Janeiro,Estado do Rio de Janeiro,20230-070,Brasil,http://www.rioscenarium.art.br,None,+55 21-3147-9000,4.5
99998,Me senti enganada...,152745909,1062290,LisianeCohem,Fui por indica\u00E7\u00E3o e com boa expectat...,pt,True,Porto Alegre. RS,21/02/2013,None,...,Celeiro,Rua Dias Ferreira 199,Rio de Janeiro,Estado do Rio de Janeiro,22431-050,Brasil,http://www.celeiroculinaria.com.br,celeiro@celeiroculinaria.com.br,55 21 2274 7843,3.5
99999,Restaurante excelente,649755498,12433911,brunotulio,Experi\u00EAncia fant\u00E1stica com comida de...,pt,True,Manaus. AM,02/02/2019,02/2019,...,Coco Bambu Manaus,Avenida Coronel Teixeira. 5705 Ponta Negra. Sh...,Manaus,Rio Amazonas,69037-000,Brasil,http://www.facebook.com/cocobambumanausoficial/,rh.manaus@cocobambu.com,+55 92 3667-7430,5.0


In [23]:
_train_data = train_data.reviewBody.astype("string")
type(_train_data)
_train_data

0        Comida maravilhosa. lugar do verdadeiro chicle...
1        Comida boa. drinks legais e um ambiente bem ac...
2        Excelente pra ir com a fam\u00EDlia! \u00D3tim...
3        Tomo sorvete na Visabor faz mais de 30 anos. S...
4        O melhor hamburger que j\u00E1 comi! Excelente...
                               ...                        
99995    Ainda n\u00E3o inaugurado oficialmente. est\u0...
99996    Bom atendimento. lugar bonito. comida muito go...
99997    O Rio Scenarium \u00E9 uma das baladas mais fa...
99998    Fui por indica\u00E7\u00E3o e com boa expectat...
99999    Experi\u00EAncia fant\u00E1stica com comida de...
Name: reviewBody, Length: 100000, dtype: string

In [ ]:
# _blob = TextBlob(_train_data))
# print(_blob.translate(to='en'))

In [24]:
data = train_data
X = data.iloc[:, 4].values # aqui pode ser selecionado a coluna

In [25]:
_X = X
print(_X)

['Comida maravilhosa. lugar do verdadeiro chiclete de camar\\u00E3o. \\nO gar\\u00E7om \\u00C9ric Lima nos atendeu muito bem. pretendemos voltar assim que poss\\u00EDvel.'
 'Comida boa. drinks legais e um ambiente bem aconchegante. \\nProvamos muitas coisas e tudo nos agradou. Principalmente um pato servido em um dos pratos da casa. sensacional!'
 'Excelente pra ir com a fam\\u00EDlia! \\u00D3tima Comida. ambiente agrad\\u00E1vel. bom atendimento e estacionamento. Vale a pena conhecer'
 ...
 'O Rio Scenarium \\u00E9 uma das baladas mais famosas do Rio de Janeiro. \\u00C9 bem grande e lotada de estrangeiros. A bebida n\\u00E3o \\u00E9 t\\u00E3o cara como em outras cidades. por exemplo. S\\u00E3o Paulo. O lado negativo \\u00E9 com certeza a fila. Se voc\\u00EA n\\u00E3o chegar cedo. vai ficar pelo menos 30 minutos esperando para entrar.'
 'Fui por indica\\u00E7\\u00E3o e com boa expectativa!\\n116.00 o kilo de salada... Bem caro. n\\u00E9?\\nO alface tava murcho. nada de op\\u00E7\\u00F5

In [ ]:
# test = np.array_str(X).encode('utf-16', 'surrogatepass').decode('utf-16')
# serieTest = pd.Series(test)
# print(serieTest)

In [27]:
serie =  pd.Series(np.array_str(X))
serie

0    ['Comida maravilhosa. lugar do verdadeiro chic...
dtype: object

In [28]:
blob = TextBlob(serie[0])
print(blob.translate(to='en'))

['Wonderful food. place of the real gum of shrimp \\ u00E3o. \\ nThe gar \\ u00E7om \\ u00C9ric Lima served us very well. we plan to return as soon as possible. '
 'Nice food. cool drinks and a very cozy atmosphere. \\ nWe proved a lot of things and everything pleased us. Especially a duck served in one of the house dishes. sensational!'
 'Excellent to go with the family! \\ u00D3tima Comida. pleasant environment. good service and parking. Worth knowing'
 ...
 'The Rio Scenarium \\ u00E9 one of the most famous ballads in Rio de Janeiro. \\ u00C9 very large and crowded with foreigners. The drink is not as expensive as in other cities. for example. S \\ u00E3o Paulo. The downside \\ u00E9 is definitely the queue. If you don't arrive early. you'll be waiting at least 30 minutes to get in. '
 'I went by referral \\ u00E7 \\ u00E3o and with good expectations! \\ n116.00 the kilo of salad ... Very expensive. n \\ u00E9? \\ nThe lettuce was withered. no seasoning options \\ u00E7 \\ u00F5es. 

In [ ]:
dic2 = pd.DataFrame(train_data)
dic2['reviewBody']
count = 0
for x in dic2['reviewBody']:
    count = count + 1  
    print(type(x))
    print(count) #vai imprimir todos os reviews
    blob = TextBlob(x)
    print(blob.translate(to='en'))
#     if count == 5:
#         break

<class 'str'>
1
Wonderful food. place of the real shrimp gum. \ nThe waiter Lima served us very well. we plan to return as soon as possible.
<class 'str'>
2
Nice food. cool drinks and a very cozy atmosphere. \ nWe proved a lot of things and everything pleased us. Especially a duck served in one of the house dishes. sensational!
<class 'str'>
3
Excellent to go with the family! \ u00D3tima Comida. pleasant environment. good service and parking. worth knowing
<class 'str'>
4
I have had ice cream at Visabor for over 30 years. Fruit or artificial flavors. Crunchy cone and simple atmosphere. Very good prices. I recommend
<class 'str'>
5
The best hamburger I've ever eaten! Excellent service. the food was fine. The restaurant is quite small. but pleasant and fun.
<class 'str'>
6
The restaurant is new. super cozy and charming with a \ u00F3timo service. The portions are generous. very tasty and at a fair price. Good selection of wines. I highly recommend the seafood spaghetti. Bolognese. carbon

Vegetarian price restaurant with the right to repeat food and juices. but I felt full right on the first plate and juice. Accepts credit card. There is a lack of dishes with lighter seasoning. more colorful and less cloying aroma. Going without eating meat is not easy. if vegetarian dishes are not tasty it is difficult to return.
<class 'str'>
31
Place with typical food and specialized in carne de sol. which is accompanied by string beans. manioc. pirirúoo de leite ..... A plate for two and left over. in our case. Fair price and excellent service. It also serves goat meat.
<class 'str'>
32
I have lunch in January this year! Main course \ u00E9 \ u00E0 Carne do Sol \ u00E0 Potiguar. being very tender meat and accompanies: Cooked Macaxeira. Pa \ u00E7oca. Butter from the bottle. Crumbs. String beans .....!
<class 'str'>
33
It \ u00C9 a good restaurant that. to last. more varieties are missing. Inside the hotel is a talented singer and everyone is offered. who sings well.
<class 'str'>
34

The place is simple. but the food is good and reasonably priced. It is possible to order and take it home.
<class 'str'>
68
I learned that it was one of the best. so I wanted to take the test! Great! I ordered cod and Pastel de Bel \ u00E9m, which left something to be desired from what I ate in Portugal! Added to a good wine and good chat! I recommend!
<class 'str'>
69
The best late afternoon snack in Porto Velho. Regional and Pará food. where you have the best crab soup. \ nSugest: the perfect tempting cake
<class 'str'>
70
A card of great natural and tasty options. well executed and with an excellent service. Every time I eat at Frutaria. I keep thinking that I have to go back to try another dish.
<class 'str'>
71
The place is located in a beautiful Vilela in Petr \ u00F3polis. and has external and internal tables. I went to the night. the service was good and fast. I ordered a divine shrimp crepe (R $ 22) and my husband ordered a filet mignon (R $ 35). The \ u00FAnica thing that lef

In [ ]:
result = []
x = []
for _data in data:
  print(type(_data), _data)
  if _data == 'reviewBody':    
    for x in _data:
      print()
    print(result)

In [ ]:
for x in serie:
  print(x)
  blob = TextBlob(x)
  print(blob.translate(to='en'))

In [ ]:
train_data

In [ ]:
def limparDados(text):
    text = re.sub()
    return text

In [ ]:
X

In [ ]:
_X = np.array_str(X)
print(_X)
#_X.decode('unicode-escape')
X.shape #Formato 

In [ ]:
type(X) # tipo do dado

In [ ]:
data_str = _X[0:10000]
str(data_str).encode('unicode_escape').decode('unicode_escape')

In [ ]:
def decodeUnicodeArray(_string):
    espacoSeq = _string.group(0)
    digito = espacoSeq[2:]
    ordinal = int(digito, 16)
    char = chr(ordinal)
    return char
    

In [ ]:
re.sub(r'(\\u[09A-Fa-f]+)', decodeUnicodeArray, "test")

In [ ]:
train_data.describe()

In [ ]:
#data.drop(['title', 'restaurantId','author','reviewBody','language','userLocation','publicationDate','visitDate','ratingValue','itemReviewed','address','addressLocality','addressRegion','postalCode','country','website','email','phoneNumber','averageRating'], axis = 1, inplace=True)

In [ ]:
#train_data.mobileDevice.unique()

In [ ]:
sns.countplot(train_data.mobileDevice);

In [ ]:
sns.catplot(x="mobileDevice", kind="count", palette="ch:.25", data=train_data)

In [ ]:
%pwd

In [ ]:
f1 = (2*66.6*66.6)/(66.6+66.6)

In [ ]:
f1

In [ ]:
f2 = (2*60*100)/(60+100)
f2


In [ ]:
f1p = (2*50*50)/(50+50)
f1p

In [ ]:
P = 4/4
R = 4/4
f2p = (2*P*R)/(P+R)
f2p

In [ ]:
print('Conhe\u00E7o e aprecio comida \u00E1rabe desde muito pequena. Minha av\u00F3 cozinhava divinamente. \nRaras vezes degustei fora de casa sabores t\u00E3o marcantes e harm\u00F4nicos! O ambiente \u00E9 familiar e iluminado. O novo espa\u00E7o ficou incr\u00EDvel. Frequento e Recomendo! \u2764\uFE0F\nParab\u00E9ns.  \uD83D\uDC4F\uD83C\uDFFB\uD83D\uDC4F\uD83C\uDFFB\uD83D\uDC4F\uD83C\uDFFB'.encode('utf-16', 'surrogatepass').decode('utf-16'))
print(type('Conhe\u00E7o e aprecio comida \u00E1rabe desde muito pequena. Minha av\u00F3 cozinhava divinamente. \nRaras veze'))
# import nltk
# nltk.download('punkt')
# from textblob import TextBlob
#text = 'Conhe\u00E7o e aprecio comida \u00E1rabe desde muito pequena. Minha av\u00F3 cozinhava divinamente. \nRaras vezes degustei fora de casa sabores t\u00E3o marcantes e harm\u00F4nicos! O ambiente \u00E9 familiar e iluminado. O novo espa\u00E7o ficou incr\u00EDvel. Frequento e Recomendo! \u2764\uFE0F\nParab\u00E9ns.  \uD83D\uDC4F\uD83C\uDFFB\uD83D\uDC4F\uD83C\uDFFB\uD83D\uDC4F\uD83C\uDFFB \nFoi muito ruim demais! \n Não irei voltar mais neste lugar'.encode('utf-16', 'surrogatepass').decode('utf-16')
text = 'Sushiloko j\u00E1 foi melhor e mais barato. claro que continua bom. mas os pre\u00E7os dos combos j\u00E1 n\u00E3o est\u00E3o mais em conta'.encode('utf-16', 'surrogatepass').decode('utf-16')
comentario_blob = TextBlob(text)
comentario_blob_us = comentario_blob.translate(to='us')

for sentence_br, sentence in zip(comentario_blob.sentences, comentario_blob_us.sentences):
    print(sentence_br, ':', sentence.sentiment.subjectivity)

In [ ]:
for sentence_br, sentence in zip(comentario_blob.sentences, comentario_blob_us.sentences):
    print(sentence_br, ':', sentence.sentiment.polarity)